# dbt (data build tool)
[dbt](https://docs.getdbt.com/docs/introduction) does not extract or load data, but it’s state-of-the-art at transforming data that’s already available in a database — dbt does the T in ELT (Extract, Load, Transform) processes.

---


## Set-up
### PostgreSQL
#### Start PostgreSQL database using the controlboard
Let's assume our raw data lives in a PostgreSQL database. Follow the instructions in the Notebook [Controlboard](controlboard.ipynb). 

#### Create a new table for dbt in PostgreSQL
<div class="alert alert-block alert-info">
<b>Note:</b> To run this code, make sure you select "Python 3 (ipkernel)" (Kernel -> Change Kernel)
</div>

Create a new database for this tutorial:

In [1]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy_utils import database_exists, create_database
from urllib import parse

In [4]:
# Name of your database - this database does NOT exist yet (create it below with `create_database()`)
database = 'tutorial'
username = 'dbuser'
password = 'rA4I2WYECsV4bqbw'

# Connection details according to docker-compose.yml - do not change this
dialect = 'postgresql'  # Could be almost any other DB technology
host = 'postgresql.myproject.svc.cluster.local'  # Name of the Kubernetes service
port = 5432

# URL-encode password for characters like %, ä, ...
password = parse.quote_plus(password)

url = f'{dialect}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

In [5]:
if not database_exists(engine.url):
    create_database(engine.url)

print(f'Database "{database}" exists: {database_exists(engine.url)}')

Database "tutorial" exists: True


### Init dbt
We are using the Terminal within Jupyter since dbt expects some answers using the command prompt - this is impossible to implement using Jupyter cells here :-(. 

1. In Jupyter, hit the blue "+" sign at the top left and start a new Terminal
2. Create a new folder for dbt projects, then navigate to it:
```console
mkdir -p ~/work/dbt
cd ~/work/dbt
```
3. Tell the Terminal that you want to use the Python environment `dbt` instead of the standard `base` one (otherwise you'll get an error "bash: dbt: command not found"):
```console
(base) jovyan@...:~/$ conda activate dbt
(dbt) jovyan@...:~/$ 
```

4. Initialize dbt to create a new project called e.g. `tutorial` an entire new folder structure for your dbt project:
```console
dbt init tutorial
```
5. Answer dbt's questions e.g. like this:
    * Enter a name for your project (letters, digits, underscore): `tutorial`
    * Which database would you like to use? `1`
6. dbt saves connection infos by default to `~/.dbt/profiles.yml`. This is not useful since that file will be lost quickly if our Jupyter pod get deleted. Let's safe settings to e.g. `/data/dbt`:
```console
$ mkdir ~/data/dbt
```
7. Using Jupyter's file browser on the left, navigate to `/data/dbt`, right click into the empty large field below "Name" and create a new file called `profiles.yml`. Double click to open it.
8. Paste the following. Don't forget to set a password; the rest should be fine as-is.
```yaml
tutorial:
  outputs:

    dev:
      type: postgres
      threads: 1
      host: postgresql.myproject.svc.cluster.local
      port: 5432
      user: dbuser
      pass: rA4I2WYECsV4bqbw
      dbname: tutorial
      schema: public

  target: dev

```
9. Test your set-up using the terminal again:
```console
$ cd /home/jovyan/work/dbt/tutorial
$ dbt debug --profiles-dir /home/jovyan/data/dbt
```

## Set-up 

Make sure that you can view hidden Linux files: go to View -> "Show Hidden Files"
<div class="alert alert-block alert-info">
<b>Note:</b> Jupyter's "Show Hidden Files" is currently broken - you won't see all dbt files. Either edit any dbt files on your host computer or use the Jupyter command prompt.
</div>

## Set-up git
Todo

## Extract and load
Switch kernel from `dbt` to `Python 3 (ipkernel)`

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy_utils import database_exists, create_database
from urllib import parse

import pandas as pd
import requests
import json

raw_data_url = 'https://github.com/openZH/covid_19/raw/master/fallzahlen_kanton_alter_geschlecht_csv/COVID19_Fallzahlen_Kanton_ZH_altersklassen_geschlecht.csv'
coingecko = 'https://api.coingecko.com/api/v3'
endpoint = '/coins/ethereum/market_chart'
params = {
    'vs_currency': 'CHF',
    'days': 90,
    'interval': 'hourly'
}

In [ ]:
# Name of your database - this database does NOT exist yet (create it below with `create_database()`)
database = 'tutorial'
username = 'dbuser'
password = '662VZUE5RJJG2puI'

# Connection details according to docker-compose.yml - do not change this
dialect = 'postgresql'  # Could be almost any other DB technology
host = 'postgresql.myproject.svc.cluster.local'  # Name of the Kubernetes service
port = 5432

# URL-encode password for characters like %, ä, ...
password = parse.quote_plus(password)

url = f'{dialect}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

In [ ]:
if not database_exists(engine.url):
    create_database(engine.url)

print(f'Database "{database}" exists: {database_exists(engine.url)}')

In [ ]:
df = pd.read_csv(raw_data_url)
df

In [ ]:
r = requests.get(coingecko + endpoint, params=params)
r

In [ ]:
df = json.loads(r.text)
df = df['prices']
df = pd.DataFrame(df, columns=['time', 'value'])

In [ ]:
df['time'] = pd.to_datetime(df['time'] * 1000000)

In [ ]:
df

In [ ]:
df.to_sql(
    'raw_data',  # table name
    con=engine,
    if_exists='replace',
    index=False,  # In order to avoid writing DataFrame index as a column
)

In [ ]:
df2 = pd.read_sql('SELECT * FROM raw_data', con=engine)

In [ ]:
df2

## Airports and flights

In [ ]:
raw_data = (
    ('airports', 'https://github.com/hgrif/dbt_tutorial/raw/master/flights_data/airports.csv'),
    ('carriers', 'https://github.com/hgrif/dbt_tutorial/raw/master/flights_data/carriers.csv'),
    ('flights', 'https://github.com/hgrif/dbt_tutorial/raw/master/flights_data/flights.csv')
)
for table, url in raw_data:
    df = pd.read_csv(url)
    df.to_sql(table, con=engine, if_exists='replace')